In [ ]:
import pandas as pd
import pickle
from sortabilitytime.sortnregress_time import (
    var_sort_regress,
    r2_sort_regress_ts,
    r2_sort_regress,
    random_sort_regress_ts,
    random_sort_regress,
    var_sort_regress_ts_reverse,
    var_sort_regress_reverse,
    var_sort_regress,
    var_sort_regress_ts,
    ts_graph_to_summary_graph,
)
from sortabilitytime.sortability_ts import var_sortability, r2_sortability
from sortabilitytime.dynotears import Dynotears
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr
import tigramite.data_processing as pp
from tigramite.independence_tests.parcorr import ParCorr
import numpy as np
import warnings
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [2]:
rivers = [
    "danube",
    "lower-colorado",
    "middle-colorado",
    "upper-colorado",
]


In [10]:
results = pd.DataFrame()

In [4]:
rivers

['danube', 'lower-colorado', 'middle-colorado', 'upper-colorado']

In [3]:

parcorr = ParCorr(
    significance="analytic",
    #                   mask_type='y'
)

In [ ]:
r2s = []
vars = []
f1s_var_sortnregress = []
f1s_var_sortnregress_reverse = []
f1s_r2_sortnregress = []
f1s_random = []
f1s_pcmci = []

tau_max = 4

for river in tqdm(rivers):
    data_file = f"../Data/River/{river}/data.pk" 
    with open(data_file, "rb") as file_name:
        obj = pickle.load(file_name)
        df = pd.DataFrame(obj["data"], columns=obj["columns"])
        labels = obj["labels"]
        G_true = obj["G_true"]
        df = df.dropna()
        vars.append(var_sortability(df.to_numpy(), G_true))

        r2s.append(r2_sortability(df.to_numpy(), G_true))
        data = df.to_numpy()
        dataframe = pp.DataFrame(data)
        pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=0)
        results_pcmci = pcmci.run_pcmciplus(tau_max=3, pc_alpha=0.01)
        results_pcmci["p_matrix"] = np.where(
        results_pcmci["p_matrix"] < 0.01, results_pcmci["p_matrix"], 0)
        a_pcmci = results_pcmci["p_matrix"]
        a_pcmci[a_pcmci != 0] = 1

        a_pcmci = ts_graph_to_summary_graph(a_pcmci)

        a_est_var_sortnregress = var_sort_regress_ts(data, tau_max=tau_max)

        a_est_var_sortnregress_reverse = var_sort_regress_ts_reverse(data, tau_max=tau_max)
        a_est_r2_sortnregress = r2_sort_regress_ts(data, tau_max=tau_max)
        a_est_var_sortnregress[:, :, 0] = var_sort_regress(data)
        a_est_var_sortnregress_reverse[:, :, 0] = var_sort_regress_reverse(data)
        a_est_r2_sortnregress[:, :, 0] = r2_sort_regress(data)
        a_est_random = random_sort_regress_ts(data, tau_max=tau_max)
        a_est_random[:, :, 0] = random_sort_regress(data)

        a_est_var_sortnregress = ts_graph_to_summary_graph(a_est_var_sortnregress)

        a_est_var_sortnregress_reverse = ts_graph_to_summary_graph(a_est_var_sortnregress_reverse)

        a_est_r2_sortnregress = ts_graph_to_summary_graph(a_est_r2_sortnregress)

        a_est_random = ts_graph_to_summary_graph(a_est_random)

        # replace all values above !=0 with 1
        a_est_var_sortnregress[a_est_var_sortnregress != 0] = 1
        a_est_r2_sortnregress[a_est_r2_sortnregress != 0] = 1
        a_est_random[a_est_random != 0] = 1
        f1_var_sortnregress = f1_score(
            G_true.flatten(), a_est_var_sortnregress.flatten()
        )

        f1_var_sortnregress_reverse = f1_score(
            G_true.flatten(), a_est_var_sortnregress_reverse.flatten()
        )
        f1_r2_sortnregress = f1_score(G_true.flatten(), a_est_r2_sortnregress.flatten())
        f1_random = f1_score(G_true.flatten(), a_est_random.flatten())
        f1_pcmci = f1_score(G_true.flatten(), a_pcmci.flatten())

        f1s_var_sortnregress.append(f1_var_sortnregress)
        f1s_r2_sortnregress.append(f1_r2_sortnregress)
        f1s_random.append(f1_random)
        f1s_pcmci.append(f1_pcmci)
        f1s_var_sortnregress_reverse.append(f1_var_sortnregress_reverse)


# results["auc_dynotears"] = aucs_dynotears

100%|██████████| 4/4 [00:05<00:00,  1.49s/it]


In [11]:
results["river"] = rivers
results["var-sortability"] = vars
results["$r^2$-sortability"] = r2s
results["f1_var_sortnregress"] = f1s_var_sortnregress
results["f1_var_sortnregress_reverse"] = f1s_var_sortnregress_reverse
results["f1_r2_sortnregress"] = f1s_r2_sortnregress
results["f1_random"] = f1s_random
results["f1_pcmci"] = f1s_pcmci

In [12]:
results

# round 4 decimals
results = results.round(2)


results

,river,var-sortability,$r^2$-sortability,f1_var_sortnregress,f1_var_sortnregress_reverse,f1_r2_sortnregress,f1_random,f1_pcmci
0,danube,0.07,0.82,0.08,0.19,0.17,0.12,0.56
1,lower-colorado,0.35,0.61,0.09,0.08,0.08,0.11,0.36
2,middle-colorado,0.29,0.94,0.10,0.23,0.26,0.15,0.26
3,upper-colorado,0.43,0.95,0.26,0.06,0.29,0.24,0.41
